In [1]:
import pandas as pd

# Настройка для правильного отображения дробной части чисел
pd.set_option("display.precision", 15)

# Журналы  Scopus Q1 и Q2 по SNIP с учётом методики «Приоритет 2030»

Основная цель данной заметки проиллюстрировать методы и механики, которые позволят университетам отфильтровать из общего перечня журналов индексируемых в Scopus, журналы, написания статей в которых позволит выполнять показатели государственной программы поддержки университетов Российской Федерации — [«Приоритет 2030»](https://priority2030.ru/).

## Нормативнык документы

Методика расчёта целевых показателей эффективности описана в [Приказе Министерства науки и высшего образования Российской Федерации от 31.05.2021 № 432](http://publication.pravo.gov.ru/Document/View/0001202106230024?index=0&rangeSize=1). Нас интересует один показатель Р2(с1) —Количество публикаций, индексируемых в базе данных Scopus и отнесенных к I и II квартилям SNIP, в расчете на одного НПР. Методика расчёт данного показателя описан следующм образом:

> Отношение числа публикаций университета, определенное фракционным (дробным) счетом по организациям, за отчетный год и два года, предшествующих отчетному, в научных журналах I и II квартилей (по величине показателя Source Normalized Impact per Paper), индексируемых в базе данных Scopus, к средней списочной численности НПР за отчетный год.
> Расчет ведется по данным аналитической системы SciVal, справочно на 1 января года, следующего за отчетным, фактически на 1 июля года, следующего за отчетным. Учитываются публикации типов "Article", "Review" в журналах ("Journal"), которые не включены в список источников, индексация которых прекращена.
> Значение SNIP должно иметь 95% достоверности по данным CWTS (https://journalindicators.com/). Учитываются только публикации, привязанные к верифицированному профилю организации в базе данных Scopus.
> Для компьютерных наук дополнительно учитываются публикации типа “Conference Proceeding”, сделанные на конференциях уровня А* в области компьютерных наук (список конференций уровня А* в компьютерных науках по рейтингу CORE (версия 2020 года) приведен в приложении к настоящему перечню).
> Из учета исключаются публикации "Article in Press".
> Совместные публикации учитываются на основе метода фракционного (дробного) счета. Если у статьи несколько авторов, то балл публикации делится поровну между авторами. Если авторы публикации аффилированы с более чем одной организацией, то балл автора делится поровну между аффилиациями. Университет получает балл за публикацию, равный сумме баллов всех авторов с его аффилиацией.

## Используемые исходные данные

Все ниже приведённые данные **акутальны на момент написания данного материала** — 15 декабря 2021 года.

* Выгрузка списка источников и показателей Scopus, скачанная со [страницы Scopus](https://www.scopus.com/sources.uri?zone=TopNavBar&origin=searchbasic) (проиллюстрированно ниже на gif-анимации). Компания Elsevier предоставляет данные в виде файла с расширением xlsb и начиная с 2011 года. Для данного материала я ограничился данными 2020 года и для удобства работы экспортировал из в файл csv.

![](../assets/images/scopus-download-sources.gif)

* Компания Elsevier для упрощения жизни университетам и аналитикам создала специальную [страницу посвящённую прорамме «Приоритет 2030»](https://elsevierscience.ru/info/programma-prioritet-2030/), на которой опубликовала [«Пороговые значения SNIP для 1%, 5%, 10%, 25%, 50%, 75%»](https://elsevierscience.ru/files/SNIP_Thresholds_25Jun2021.xlsx). Для удобноства данные так же переведены в [формат csv](https://github.com/safonovpro/articles-priority-2030/blob/master/data/snip-thresholds.csv)
* Также нам понадобится данные Центра научных и технологических исследований Лейденского университета, для определения «95% достоверности», которые можно скачать с [данной страницы](https://www.journalindicators.com/downloads). Исходный файл имеет формат xlsx, который для удобства работы я заменил на [csv-файл](https://github.com/safonovpro/articles-priority-2030/blob/master/data/cwts-journal-indicators.csv).

### Подготовка исходных данных для работы

Приведём в порядок данные от Scopus, оставим только необходимое и посмотрим на первые 10 строк:


In [2]:
scopus = pd.read_csv('../data/scopus-sources-2020.csv', dtype={
    "Print ISSN": str,
    "E-ISSN": str
}).fillna({
    "Print ISSN": '-',
    "E-ISSN": '-'
})[['Scopus Source ID', 'Title', 'SNIP', 'Print ISSN', 'E-ISSN']].drop_duplicates(subset=['Scopus Source ID']).reset_index(drop=True)
scopus['Print ISSN'] = scopus['Print ISSN'].apply(lambda x: x if x == '-' or len(x) >= 8 else '0' * (8 - len(x)) + x)
scopus['E-ISSN'] = scopus['E-ISSN'].apply(lambda x: x if x == '-' or len(x) >= 8 else '0' * (8 - len(x)) + x)
scopus.head(10)

,Scopus Source ID,Title,SNIP,Print ISSN,E-ISSN
0,12001,Journal of the Experimental Analysis of Behavior,1.104,00225002,19383711
1,12002,Journal of the History of the Behavioral Sciences,1.215,00225061,15206696
2,12004,Journal of Trauma and Dissociation,1.058,15299732,15299740
3,12005,Journal of Traumatic Stress,1.351,08949867,15736598
4,12006,Journal of Vocational Behavior,2.800,00018791,10959084
5,12008,Teruleti Statisztika,0.694,00187828,20648251
6,12010,Annual Review of Psychology,10.632,00664308,15452085
7,12013,Revue des Etudes Juives,0.000,04848616,1783175X
8,12014,Anuario de Psicologia,0.252,00665126,-
9,12016,Canadian Journal of Fisheries and Aquatic Scie...,1.085,0706652X,12057533


Далее подготовим данные от Лейденского университета и посмотрим на них:

In [3]:
cwts = pd.read_csv('../data/cwts-journal-indicators.csv', dtype={
    "Print ISSN": str,
    "Electronic ISSN": str,
})[['Source title', 'Print ISSN', 'Electronic ISSN', 'SNIP', 'SNIP (lower bound)', 'Source type', 'Year']]
cwts = cwts[(cwts['Source type'] == 'Journal') & (cwts['Year'] == 2020)].drop(columns=['Source type', 'Year']).reset_index(drop=True)
cwts.head(10)

,Source title,Print ISSN,Electronic ISSN,SNIP,SNIP (lower bound)
0,1700-tal: Nordic Journal for Eighteenth-Centur...,-,-,0.000000000000000,0.000000000000000
1,2D Materials,-,2053-1583,1.402114501064720,1.264999705773590
2,3 Biotech,2190-572X,2190-5738,0.993577332517916,0.920931816089613
3,3D Printing and Additive Manufacturing,2329-7662,2329-7670,1.360014718156880,0.974910682486529
4,3D Research,-,2092-6731,1.196988391345570,0.913153657740477
5,"3L: Language, Linguistics, Literature",0128-5157,2550-2247,1.224709099508030,0.911434584429030
6,4OR,1619-4500,1614-2411,1.242297207499040,0.847410262619304
7,A + U-Architecture and Urbanism,0389-9160,-,0.000000000000000,0.000000000000000
8,A Contrario,1660-7880,1662-8667,0.000000000000000,0.000000000000000
9,a/b: Auto/Biography Studies,2151-7290,-,1.156233776121340,0.598716035195324


## Получаем данные для работы

Чтобы всё работало быстрее при объединении данных двух таблиц `scopus` и `cwts`, разделим данные по значениям `SNIP` и `SNIP (lower bound)` в две разных таблицы и построим их индексы по кодам `Print ISSN` и `Electronic ISSN`:

In [4]:
issn_data = pd.DataFrame(columns=['snip', 'lower_bound'])
eissn_data = pd.DataFrame(columns=['snip', 'lower_bound'])

for i, cwts_row in cwts.iterrows():
    issn_value = cwts_row['Print ISSN'].replace('-', '')
    eissn_value = cwts_row['Electronic ISSN'].replace('-', '')

    if issn_value != '' and issn_value not in issn_data.index:
        issn_data.loc[issn_value] = {
            "snip": cwts_row['SNIP'],
            "lower_bound": cwts_row['SNIP (lower bound)']
        }

    if eissn_value != '' and eissn_value not in eissn_data.index:
        eissn_data.loc[eissn_value] = {
            "snip": cwts_row['SNIP'],
            "lower_bound": cwts_row['SNIP (lower bound)']
        }

In [5]:
issn_data.head(10)

,snip,lower_bound
2190572X,0.993577332517916,0.920931816089613
23297662,1.360014718156880,0.974910682486529
01285157,1.224709099508030,0.911434584429030
16194500,1.242297207499040,0.847410262619304
03899160,0.000000000000000,0.000000000000000
16607880,0.000000000000000,0.000000000000000
21517290,1.156233776121340,0.598716035195324
13028324,0.429876305632236,0.216193767826047
01715410,0.058641975308642,0.000000000000000
07434618,1.537999071851700,1.117962628886490


In [6]:
eissn_data.head(10)

,snip,lower_bound
20531583,1.402114501064720,1.264999705773590
21905738,0.993577332517916,0.920931816089613
23297670,1.360014718156880,0.974910682486529
20926731,1.196988391345570,0.913153657740477
25502247,1.224709099508030,0.911434584429030
16142411,1.242297207499040,0.847410262619304
16628667,0.000000000000000,0.000000000000000
14773848,1.537999071851700,1.117962628886490
18449166,1.138377187201150,0.966130967383598
15597776,0.604504601907020,0.414523869746992


Далее нам необходимо применить полученные структуры, для обогащения исходных данных Scopus, где:

* **cwts_snip** — значение SNIP с большим количеством знаков после запятой
* **cwts_lower_bound** — значение SNIP удовлетворяющее *«95% достоверности»* из методики

In [7]:
scopus['cwts_snip'] = 0
scopus['cwts_lower_bound'] = 0

for i, scopus_row in scopus.iterrows():
    issn_value = scopus_row['Print ISSN']
    issn_list = issn_value.split(' ') if issn_value != '-' else []
    eissn_value = scopus_row['E-ISSN']
    eissn_list = eissn_value.split(' ') if eissn_value != '-' else []
    found_in_issn = False

    for issn in issn_list:
        if issn in issn_data.index:
            scopus.loc[i, 'cwts_snip'] = issn_data.loc[issn, 'snip']
            scopus.loc[i, 'cwts_lower_bound'] = issn_data.loc[issn, 'lower_bound']
            found_in_issn = True
            break

    if ~found_in_issn:
        for eissn in eissn_list:
            if eissn in eissn_data.index:
                scopus.loc[i, 'cwts_snip'] = eissn_data.loc[eissn, 'snip']
                scopus.loc[i, 'cwts_lower_bound'] = eissn_data.loc[eissn, 'lower_bound']
                break

scopus.head(10)

,Scopus Source ID,Title,SNIP,Print ISSN,E-ISSN,cwts_snip,cwts_lower_bound
0,12001,Journal of the Experimental Analysis of Behavior,1.104,00225002,19383711,1.104171173126670,0.883270171837390
1,12002,Journal of the History of the Behavioral Sciences,1.215,00225061,15206696,1.215310657962800,0.621836776443208
2,12004,Journal of Trauma and Dissociation,1.058,15299732,15299740,1.057836649303670,0.762359841431676
3,12005,Journal of Traumatic Stress,1.351,08949867,15736598,1.350698523834130,1.158925521492220
4,12006,Journal of Vocational Behavior,2.800,00018791,10959084,2.800203567140800,2.391747935574450
5,12008,Teruleti Statisztika,0.694,00187828,20648251,0.694195102879223,0.460177645474752
6,12010,Annual Review of Psychology,10.632,00664308,15452085,10.631974196653900,8.585659616132149
7,12013,Revue des Etudes Juives,0.000,04848616,1783175X,0.000000000000000,0.000000000000000
8,12014,Anuario de Psicologia,0.252,00665126,-,0.251792301023991,0.086438007782370
9,12016,Canadian Journal of Fisheries and Aquatic Scie...,1.085,0706652X,12057533,1.084503475669520,0.978189484517752


Напоследок используя данные Elsevire про границы квартильности за 2020 год создадим столбец `quartile` в данных Scopus с учётом следующего:

* Q1 — для журналов `cwts_lower_bound` больше или равен `threshvalue25`;
* Q1* — для журналов не вошедших в `Q1`, где `cwts_snip` больше или равен `threshvalue25`;
* Q2 — для журналов не вошедших в `Q1` или `Q1*`, где `cwts_lower_bound` больше или равен `threshvalue50`;
* Q2* — для журналов не вошедших в `Q1`, `Q1*`, `Q2`, где `cwts_snip` больше или равен `threshvalue50`;
* Other — для всех остальных журналов

In [9]:
thresholds = pd.read_csv('../data/snip-thresholds.csv')[['year', 'threshvalue25', 'threshvalue50']]
year = 2020
q1_threshold = thresholds[thresholds['year'] == year]['threshvalue25'].values[0]
q2_threshold = thresholds[thresholds['year'] == year]['threshvalue50'].values[0]

def get_quartile(row):
    snip = row['cwts_snip']
    lower_bound = row['cwts_lower_bound']

    if lower_bound >= q1_threshold:
        return 'Q1'
    elif snip >= q1_threshold:
        return 'Q1*'
    elif lower_bound >= q2_threshold:
        return 'Q2'
    elif snip >= q2_threshold:
        return 'Q2*'

    return 'Other'


scopus['quartile'] = scopus.apply(get_quartile, axis=1)
scopus.head(10)

,Scopus Source ID,Title,SNIP,Print ISSN,E-ISSN,cwts_snip,cwts_lower_bound,quartile
0,12001,Journal of the Experimental Analysis of Behavior,1.104,00225002,19383711,1.104171173126670,0.883270171837390,Q2
1,12002,Journal of the History of the Behavioral Sciences,1.215,00225061,15206696,1.215310657962800,0.621836776443208,Q1*
2,12004,Journal of Trauma and Dissociation,1.058,15299732,15299740,1.057836649303670,0.762359841431676,Q2*
3,12005,Journal of Traumatic Stress,1.351,08949867,15736598,1.350698523834130,1.158925521492220,Q1*
4,12006,Journal of Vocational Behavior,2.800,00018791,10959084,2.800203567140800,2.391747935574450,Q1
5,12008,Teruleti Statisztika,0.694,00187828,20648251,0.694195102879223,0.460177645474752,Other
6,12010,Annual Review of Psychology,10.632,00664308,15452085,10.631974196653900,8.585659616132149,Q1
7,12013,Revue des Etudes Juives,0.000,04848616,1783175X,0.000000000000000,0.000000000000000,Other
8,12014,Anuario de Psicologia,0.252,00665126,-,0.251792301023991,0.086438007782370,Other
9,12016,Canadian Journal of Fisheries and Aquatic Scie...,1.085,0706652X,12057533,1.084503475669520,0.978189484517752,Q2


Напоследок можно посмотреть, сколько каких журналов у нас получилось:

In [10]:
scopus.groupby(['quartile']).count().reset_index()[['quartile', 'Scopus Source ID']].rename(columns={
    "Scopus Source ID": "count"
})

,quartile,count
0,Other,13384
1,Q1,3516
2,Q1*,2749
3,Q2,2411
4,Q2*,3930
